In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, explode, array, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ChiSqSelector
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression,OneVsRest
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import dayofweek
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.functions import isnan, when, count, col, lit, sum
from pyspark.sql.functions import (to_date, datediff, date_format,month)
#Se carga el conjunto de datos

spark = SparkSession.builder.master("local").appName("Avila").config("spark.some.config.option","some-value").getOrCreate()
data = spark.read.format("csv").option("header","true").option("inferSchema", "true").load(r"avila.csv")
data
#####################################################################################################
#PRIMER PUNTO
#DESCRIPCION DEL CONJUNTO DE DATOS INICIAL
#Imprime la cantidad de registros y atributos respectivamente
#print("Registros Iniciales:",data.count(),", Atributos Iniciales:",len(data.columns))

#Tipo de los atributos
#data.printSchema()

# Se revisa si existen nulos en alguno de los atributos del dataset
#print("Cantidad de Nulos en cada atributo")
#print(data.select([count(when(isnan(c),c)).alias(c) for c in data.columns]).toPandas().head())

#Descripcion de los atributos
#print("Descripcion de los atributos")
#print(data.describe().select("Summary","F1","F2","F3","F4","F5").show())
#print(data.describe().select("Summary","F6","F7","F8","F9","F10").show())

DataFrame[F1: double, F2: double, F3: double, F4: double, F5: double, F6: double, F7: double, F8: double, F9: double, F10: double, Author: string]

In [2]:
#Se verifica la correlacion entre los atributos
#pd = data.toPandas()
#print("Correlacion entre atributos")
#print(pd.corr())

#Distribucion del atributo clasificador
#print("Distribucion del atributo clasificador")
#data.groupby("Author").count().show()
#####################################################################################################
#COMIENZA EL SEGUNDO PUNTO
#LIMPIEZA DE LOS DATOS

#Como se puede ver en los diagramas de cajas, el atributo F2 tiene datos que son demasiado atipicos
#Estos registros se eliminaran
#print("LIMPIEZA DE LOS DATOS")
data = data.filter(data.F2<350)
#print("Datos Demasiado Atipicos de F2 Eliminados:",data.count())

#print("Conversion de atributos categoricos a numericos")
indexer = StringIndexer(inputCol="Author", outputCol="AuthorNum")
data = indexer.fit(data).transform(data)
data = data.drop('Author')
#data.groupby("AuthorNum").count().show()

#Se balancea cada categoria (Entre 1000 y 2000 atributos cada una)
A = data.filter(data.AuthorNum == 0.0).sample(fraction=0.24)
F = data.filter(data.AuthorNum == 0.0).sample(fraction=0.53)
E = data.filter(col("AuthorNum") == 2.0).withColumn("dummy", explode(array([lit(x) for x in range(1)]))).drop('dummy')
I = data.filter(col("AuthorNum") == 3.0).withColumn("dummy", explode(array([lit(x) for x in range(1)]))).drop('dummy')
X = data.filter(col("AuthorNum") == 4.0).withColumn("dummy", explode(array([lit(x) for x in range(2)]))).drop('dummy')
H = data.filter(col("AuthorNum") == 5.0).withColumn("dummy", explode(array([lit(x) for x in range(2)]))).drop('dummy')
G = data.filter(col("AuthorNum") == 6.0).withColumn("dummy", explode(array([lit(x) for x in range(2)]))).drop('dummy')
D = data.filter(col("AuthorNum") == 7.0).withColumn("dummy", explode(array([lit(x) for x in range(3)]))).drop('dummy')
Y = data.filter(col("AuthorNum") == 8.0).withColumn("dummy", explode(array([lit(x) for x in range(4)]))).drop('dummy')
C = data.filter(col("AuthorNum") == 9.0).withColumn("dummy", explode(array([lit(x) for x in range(8)]))).drop('dummy')
W = data.filter(col("AuthorNum") == 10.0).withColumn("dummy", explode(array([lit(x) for x in range(17)]))).drop('dummy')
B = data.filter(col("AuthorNum") == 11.0).withColumn("dummy", explode(array([lit(x) for x in range(170)]))).drop('dummy')

#Se juntan todas las categorias balanceadas
data = A.union(B).union(C).union(D).union(E).union(F).union(G).union(H).union(I).union(W).union(Y).union(X)

#print("Conjunto Balanceado")
data.groupby("AuthorNum").count().show()
#print("Numero de Registros Dataset Limpio:",data.count(),", Atributos:",len(data.columns))
#data

+---------+-----+
|AuthorNum|count|
+---------+-----+
|      8.0| 2132|
|      0.0| 6565|
|      7.0| 2115|
|      4.0| 2088|
|     11.0| 1700|
|      3.0| 1663|
|      2.0| 2190|
|     10.0| 1513|
|      6.0| 1786|
|      5.0| 2078|
|      9.0| 1648|
+---------+-----+



In [20]:
#eliminar AuthorNum
n_data = 
data = data.drop('AuthorNum')

data




DataFrame[F1: double, F2: double, F3: double, F4: double, F5: double, F6: double, F7: double, F8: double, F9: double, F10: double]

In [3]:
#reducción de dimensionalidad

import numpy as np
from sklearn.manifold import TSNE


data.select("F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9", "F10").collect()
data_array = np.array(data.select("F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9", "F10").collect())
X_embedded = TSNE(n_components=2).fit_transform(data_array)
reduccion = X_embedded
reduccion

array([[-20.973328  ,  49.002106  ],
       [  0.37042406,  53.296223  ],
       [  0.78884166,  14.009132  ],
       ...,
       [-78.46165   ,  -7.455171  ],
       [-37.25485   , -73.87922   ],
       [-37.25485   , -73.87922   ]], dtype=float32)

In [4]:
import pandas as pd
import numpy as np
nodos = pd.DataFrame(X_embedded)
nodos.columns = ["caracteristica1","caracteristica2"]
nodos

,caracteristica1,caracteristica2
0,-20.973328,49.002106
1,0.370424,53.296223
2,0.788842,14.009132
3,-25.238632,41.097012
4,-27.419462,42.780735
...,...,...
25473,-71.698441,-6.280373
25474,-78.461647,-7.455171
25475,-78.461647,-7.455171
25476,-37.254848,-73.879219


In [5]:
nodos.to_csv('nodos.csv')

In [8]:
import numpy as np
from sklearn.neighbors import KDTree
tree = KDTree(nodos, leaf_size=2)
dist, ind = tree.query(nodos[:], k=3)
arregloFinal = []
for i in range(25478):
    for num in ind[i]:
        if num != i:
            arregloFinal.append([i,num])  


In [9]:
arreglo2 = arregloFinal
arregloFinal

[[0, 9675],
 [0, 59],
 [1, 11850],
 [1, 1817],
 [2, 9680],
 [2, 10699],
 [3, 9683],
 [3, 13400],
 [4, 9688],
 [4, 12004],
 [5, 12425],
 [5, 11299],
 [6, 9690],
 [6, 12959],
 [7, 971],
 [7, 11823],
 [8, 9693],
 [8, 1145],
 [9, 9698],
 [9, 13627],
 [10, 6221],
 [10, 6223],
 [11, 9700],
 [11, 10633],
 [12, 7653],
 [12, 5190],
 [13, 222],
 [13, 891],
 [14, 17321],
 [14, 17320],
 [15, 21028],
 [15, 21029],
 [16, 9713],
 [16, 15169],
 [17, 6937],
 [17, 6935],
 [18, 12372],
 [18, 8205],
 [19, 9720],
 [19, 12758],
 [20, 9721],
 [20, 10880],
 [21, 9724],
 [21, 13382],
 [22, 9725],
 [22, 13364],
 [23, 9727],
 [23, 14397],
 [24, 9729],
 [24, 11677],
 [25, 10012],
 [25, 12396],
 [26, 11919],
 [26, 11078],
 [27, 13673],
 [27, 12303],
 [28, 9737],
 [28, 10565],
 [29, 9740],
 [29, 1341],
 [30, 9741],
 [30, 12649],
 [31, 9742],
 [31, 13954],
 [32, 9745],
 [32, 13373],
 [33, 14036],
 [33, 18535],
 [34, 9748],
 [34, 22968],
 [35, 12640],
 [35, 577],
 [36, 9751],
 [36, 824],
 [37, 12791],
 [37, 10476],
 

In [11]:
relaciones = pd.DataFrame(arregloFinal)
relaciones.columns = ["idNodo1","idNodo2"]
relaciones

,idNodo1,idNodo2
0,0,9675
1,0,59
2,1,11850
3,1,1817
4,2,9680
...,...,...
53397,25475,23519
53398,25476,25477
53399,25476,25025
53400,25477,25476


In [12]:
relaciones.to_csv('relaciones.csv')

In [41]:
grafo = []

for i in range(25503):
    nodoUno = arregloFinal[i][0]
    nodoDos = arregloFinal[i][1]
    grafo.append([nodoUno, reduccion[nodoUno][0], reduccion[nodoUno][1], nodoDos,reduccion[nodoDos][0],reduccion[nodoDos][1]])

grafo

[[0, -1.2303458, -9.364118, 15099, -0.9937553, -9.863198],
 [0, -1.2303458, -9.364118, 15100, -0.99328464, -9.863487],
 [1, 37.9544, -11.887759, 16848, 37.883392, -11.878167],
 [1, 37.9544, -11.887759, 16847, 37.882866, -11.878057],
 [2, -2.3625116, 25.85881, 9713, -2.3624554, 25.858906],
 [2, -2.3625116, 25.85881, 5527, -1.1699909, 25.50789],
 [3, 34.68277, 18.93374, 9714, 34.682777, 18.933746],
 [3, 34.68277, 18.93374, 14750, 35.207706, 18.29666],
 [4, 12.65483, -69.47392, 429, 12.595565, -69.401886],
 [4, 12.65483, -69.47392, 10869, 12.680788, -69.36375],
 [5, 14.104929, 30.45611, 9718, 14.104868, 30.456135],
 [5, 14.104929, 30.45611, 44, 14.118782, 30.325264],
 [6, 1.5005889, 6.582607, 9720, 1.5003333, 6.582729],
 [6, 1.5005889, 6.582607, 12346, 1.5353429, 6.5852833],
 [7, 12.381506, 9.895958, 11886, 12.484806, 10.011131],
 [7, 12.381506, 9.895958, 6873, 12.10542, 10.321384],
 [8, 34.08018, -61.63835, 8, 34.08018, -61.63835],
 [8, 34.08018, -61.63835, 23374, 33.854588, -62.07703],


In [43]:
dataFinal = pd.DataFrame(grafo)
dataFinal.columns = ["idNodo1", "caracteristica1Nodo1", "caracteristica2Nodo1", "idNodo2", "caracteristica1Nodo2", "caracteristica2Nodo2"]
dataFinal

,idNodo1,caracteristica1Nodo1,caracteristica2Nodo1,idNodo2,caracteristica1Nodo2,caracteristica2Nodo2
0,0,-1.230346,-9.364118,15099,-0.993755,-9.863198
1,0,-1.230346,-9.364118,15100,-0.993285,-9.863487
2,1,37.954399,-11.887759,16848,37.883392,-11.878167
3,1,37.954399,-11.887759,16847,37.882866,-11.878057
4,2,-2.362512,25.858810,9713,-2.362455,25.858906
...,...,...,...,...,...,...
25498,12749,-15.035315,-4.767326,9360,-14.616083,-4.904032
25499,12749,-15.035315,-4.767326,7894,-14.474169,-4.925288
25500,12750,2.344267,4.046925,12721,2.275422,4.058334
25501,12750,2.344267,4.046925,10376,2.270921,4.046997


In [48]:
import pandas 
dataFinal.to_csv('dataFinal.csv')

In [22]:
#data.toPandas().to_csv('avilaPre.csv')
#import pandas as pd
#import numpy as np

#array = data.to_records()
#array
#import numpy as np

#data.select("F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9", "F10", "AuthorNum").collect()
#data_array = np.array(data.select("F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9", "F10", "AuthorNum").collect())
#type(data_array)
#len(data_array)


25544

In [27]:
#from sklearn.feature_selection import VarianceThreshold
#selector = VarianceThreshold()
#selector.fit_transform(data_array)
#len(data_array)
#selector

25544

In [ ]:
#from skfeature.function.information_theoretical_based import MRMR

#selector = MRMR.mrmr(X[train], y[train], n_selected_features=num_fea)
